In [7]:
import pandas as pd
import numpy as np
import geojson as gj

In [8]:
filePath = '../static/data/'
dispFileName = f'{filePath}disp.geojson'
file_to_load = f"{filePath}source/LA_Certificated_Dispensary_Locations.csv"

In [9]:
# Load into pandas
disp_df = pd.read_csv(file_to_load)

In [ ]:
# dispURL = 'https://data.lacity.org/resource/bunu-zsyc.geojson'
# dispDF = pd.read_json(dispURL)

# dispDF.head()

In [ ]:
disp_df.reset_index(drop = True, inplace = True)

In [ ]:
disp_df[["lat","long"]] = disp_df.LOCATION.str.split(",",expand=True)

In [ ]:
# disp_df.head()

In [ ]:
# ********************DONT RUN MORE THAN ONCE OR YOU WILL NEED TO RESTART KERNAL
disp_df["lat"] = disp_df.lat.str.slice(start=1)
disp_df["lat"] = pd.to_numeric(disp_df["lat"])

In [ ]:
# ********************DONT RUN MORE THAN ONCE OR YOU WILL NEED TO RESTART KERNAL
disp_df["long"] = disp_df.long.str.slice(stop=-1)
disp_df["long"] = pd.to_numeric(disp_df["long"])

In [ ]:
# Create a year column so we can filter this in the dashboarc
disp_df["START YEAR"] = pd.DatetimeIndex(disp_df["LOCATION START DATE"]).year

In [ ]:
# disp_df.head()

In [ ]:
# disp_df.to_json(r'../JSON Files/DISP.JSON', orient = "index")
disp_df.dtypes

In [ ]:
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=gj.Point((X["long"],
                                          X["lat"]
#                                           ,X["elev"]
                                         )),
                            properties=dict(zip_code=X["ZIP CODE"],
                                            business_name = X["BUSINESS NAME"],
                                            street_address = X["STREET ADDRESS"],
                                            location_start_date = X["LOCATION START DATE"],
                                            location_start_year = X["START YEAR"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [ ]:
data2geojson(disp_df, dispFileName)